# Day 7: useEffect Hook

## **Learning Objectives**
By the end of this lesson, students will be able to:
- Understand what side effects are in React
- Use the useEffect Hook correctly
- Control when effects run using dependency arrays
- Understand component lifecycle with useEffect
- Clean up side effects properly
- Handle timers and event listeners

---

## **Part 1: What are Side Effects?**
A side effect means anything that affects something outside the component or happens after rendering.

### **Pure Functions vs Side Effects**

**Pure Function (Predictable):**
```javascript
function add(a, b) {
  return a + b; // Always returns same result for same inputs
}
```

**Side Effect (Does something beyond returning a value):**
```javascript
function logAndAdd(a, b) {
  console.log(a, b); // Side effect: logs to console
  document.title = 'Calculator'; // Side effect: changes document
  return a + b;
}
```

### **Common Side Effects in React:**
- Fetching data from an API
- Manually changing the DOM (document.title)
- Setting up event listeners (window.addEventListener)
- Setting up timers (setTimeout, setInterval)
- Reading/writing to localStorage
- Logging data to console

**Why can't we do side effects directly in the component body?**

```javascript
// ❌ WRONG - This runs on every render!
function Component() {
  console.log('This logs too many times!');
  document.title = 'My App'; // Runs every render
  
  return <div>Hello</div>;
}
```

**Solution: Use useEffect!**

---


## **Part 2: Introduction to useEffect**

### **Basic Syntax**

```javascript
import { useEffect } from 'react';

function Component() {
  useEffect(() => {
    // Side effect code goes here
    console.log('Effect ran!');
  });
  
  return <div>Hello</div>;
}
```

### **Simple Example: Document Title**

```javascript
import { useState, useEffect } from 'react';

function Counter() {
  const [count, setCount] = useState(0);
  
  useEffect(() => {
    document.title = `Count: ${count}`;
  });
  
  return (
    <div>
      <p>Count: {count}</p>
      <button onClick={() => setCount(count + 1)}>Increment</button>
    </div>
  );
}
```

**What happens:** Every time the component re-renders, the effect runs and updates the document title.

---


## **Part 3: The Dependency Array**

### **Three Ways to Use useEffect**

**1. No Dependency Array (Runs After Every Render)**
```javascript
useEffect(() => {
  console.log('Runs after every render');
});
```

**2. Empty Dependency Array (Runs Once on Mount)**
```javascript
useEffect(() => {
  console.log('Runs only once when component mounts');
}, []); // Empty array
```

**3. With Dependencies (Runs When Dependencies Change)**
```javascript
useEffect(() => {
  console.log('Runs when count changes');
}, [count]); // Runs when count changes
```

### **Understanding Dependencies**

```javascript
function UserProfile() {
  const [userId, setUserId] = useState(1);
  const [userName, setUserName] = useState('');
  
  // Only runs when userId changes
  useEffect(() => {
    console.log(`User ID changed to: ${userId}`);
  }, [userId]);
  
  return (
    <div>
      <p>User ID: {userId}</p>
      <p>User Name: {userName}</p>
      <button onClick={() => setUserId(userId + 1)}>Next User</button>
      <input 
        value={userName}
        onChange={(e) => setUserName(e.target.value)}
        placeholder="Change name"
      />
    </div>
  );
}
```

**Note:** Changing `userName` won't trigger the effect because it's not in the dependency array.

---


## **Part 4: Component Lifecycle with useEffect**

### **Component Lifecycle Phases:**

1. **Mounting** - Component is created and added to DOM
2. **Updating** - Component re-renders due to state/props changes
3. **Unmounting** - Component is removed from DOM

### **useEffect and Lifecycle**

```javascript
function LifecycleDemo() {
  const [count, setCount] = useState(0);
  
  // Runs on mount (once)
  useEffect(() => {
    console.log('Component mounted!');
  }, []);
  
  // Runs on every update
  useEffect(() => {
    console.log('Component updated!');
  });
  
  // Runs when count changes
  useEffect(() => {
    console.log('Count changed to:', count);
  }, [count]);
  
  return (
    <div>
      <p>Count: {count}</p>
      <button onClick={() => setCount(count + 1)}>Increment</button>
    </div>
  );
}
```

---


## **Part 5: Cleanup Functions**

### **Why Cleanup?**

Some side effects need to be "cleaned up" to prevent memory leaks or unwanted behavior:
- Remove event listeners
- Cancel timers
- Close connections

### **How to Cleanup**

Return a function from useEffect:

```javascript
useEffect(() => {
  // Setup
  const timer = setInterval(() => {
    console.log('Tick');
  }, 1000);
  
  // Cleanup
  return () => {
    clearInterval(timer);
    console.log('Timer cleaned up');
  };
}, []);
```

### **When Cleanup Runs:**
- Before the effect runs again (if dependencies change)
- When component unmounts

---


## **Part 6: Practical Examples**

### **Example 1: Live Clock**

```javascript
import { useState, useEffect } from 'react';

function Clock() {
  const [time, setTime] = useState(new Date());
  
  useEffect(() => {
    // Update time every second
    const intervalId = setInterval(() => {
      setTime(new Date());
    }, 1000);
    
    // Cleanup on unmount
    return () => clearInterval(intervalId);
  }, []); // Empty array = run once
  
  return (
    <div>
      <h2>Current Time</h2>
      <p>{time.toLocaleTimeString('en-NG')}</p>
    </div>
  );
}

export default Clock;
```

### **Example 2: Auto-Save Text**

```javascript
import { useState, useEffect } from 'react';

function AutoSaveInput() {
  const [text, setText] = useState('');
  const [saveStatus, setSaveStatus] = useState('');
  
  useEffect(() => {
    if (text === '') return; // Don't save empty text
    
    setSaveStatus('Saving...');
    
    // Simulate save after 1 second
    const timer = setTimeout(() => {
      console.log('Saved:', text);
      setSaveStatus('Saved!');
    }, 1000);
    
    // Cleanup: cancel timer if text changes before 1 second
    return () => clearTimeout(timer);
  }, [text]);
  
  return (
    <div>
      <input
        value={text}
        onChange={(e) => setText(e.target.value)}
        placeholder="Type something..."
      />
      <p>{saveStatus}</p>
    </div>
  );
}

export default AutoSaveInput;
```

### **Example 3: Keyboard Event Listener**

```javascript
import { useState, useEffect } from 'react';

function KeyPressLogger() {
  const [lastKey, setLastKey] = useState('');
  const [keyCount, setKeyCount] = useState(0);
  
  useEffect(() => {
    function handleKeyPress(event) {
      setLastKey(event.key);
      setKeyCount(prev => prev + 1);
    }
    
    window.addEventListener('keypress', handleKeyPress);
    
    return () => {
      window.removeEventListener('keypress', handleKeyPress);
    };
  }, []);
  
  return (
    <div>
      <h2>Key Press Logger</h2>
      <p>Last key pressed: <strong>{lastKey || 'None'}</strong></p>
      <p>Total keys pressed: {keyCount}</p>
      <p style={{ color: '#666' }}>Press any key to test</p>
    </div>
  );
}

export default KeyPressLogger;
```

---


## **Part 7: Complete Example - Nigerian Prayer Time Tracker**

Let's build an app that shows current time and displays different greetings based on time of day.

### **PrayerTimeTracker.jsx**

```javascript
import { useState, useEffect } from 'react';
import './PrayerTimeTracker.css';

function PrayerTimeTracker() {
  const [currentTime, setCurrentTime] = useState(new Date());
  const [greeting, setGreeting] = useState('');
  const [nextPrayer, setNextPrayer] = useState('');
  
  // Prayer times (24-hour format)
  const prayerTimes = [
    { name: 'Fajr', hour: 5, minute: 30 },
    { name: 'Zuhr', hour: 13, minute: 0 },
    { name: 'Asr', hour: 16, minute: 30 },
    { name: 'Maghrib', hour: 18, minute: 45 },
    { name: 'Isha', hour: 20, minute: 0 }
  ];
  
  // Update time every second
  useEffect(() => {
    const timer = setInterval(() => {
      setCurrentTime(new Date());
    }, 1000);
    
    return () => clearInterval(timer);
  }, []);
  
  // Update greeting based on time
  useEffect(() => {
    const hour = currentTime.getHours();
    
    if (hour >= 5 && hour < 12) {
      setGreeting('Good Morning! ☀️');
    } else if (hour >= 12 && hour < 16) {
      setGreeting('Good Afternoon! 🌤️');
    } else if (hour >= 16 && hour < 20) {
      setGreeting('Good Evening! 🌅');
    } else {
      setGreeting('Good Night! 🌙');
    }
  }, [currentTime]);
  
  // Calculate next prayer
  useEffect(() => {
    const now = currentTime;
    const currentMinutes = now.getHours() * 60 + now.getMinutes();
    
    // Find next prayer
    for (let prayer of prayerTimes) {
      const prayerMinutes = prayer.hour * 60 + prayer.minute;
      if (prayerMinutes > currentMinutes) {
        const diff = prayerMinutes - currentMinutes;
        const hours = Math.floor(diff / 60);
        const minutes = diff % 60;
        setNextPrayer(`${prayer.name} in ${hours}h ${minutes}m`);
        return;
      }
    }
    setNextPrayer('Fajr tomorrow');
  }, [currentTime]);
  
  // Log when component mounts
  useEffect(() => {
    console.log('Prayer Time Tracker mounted');
    
    return () => {
      console.log('Prayer Time Tracker unmounted');
    };
  }, []);
  
  return (
    <div className="prayer-tracker">
      <h1>🕌 Prayer Time Tracker</h1>
      
      <div className="current-time">
        <h2>{currentTime.toLocaleTimeString('en-NG')}</h2>
        <p>{currentTime.toLocaleDateString('en-NG', { 
          weekday: 'long', 
          year: 'numeric', 
          month: 'long', 
          day: 'numeric' 
        })}</p>
      </div>
      
      <div className="greeting">
        <h3>{greeting}</h3>
      </div>
      
      <div className="next-prayer">
        <p>Next Prayer: {nextPrayer}</p>
      </div>
      
      <div className="prayer-times">
        <h3>Today's Prayer Times</h3>
        {prayerTimes.map((prayer, index) => (
          <div key={index} className="prayer-time">
            <span>{prayer.name}</span>
            <span>{`${prayer.hour}:${prayer.minute.toString().padStart(2, '0')}`}</span>
          </div>
        ))}
      </div>
    </div>
  );
}

export default PrayerTimeTracker;
```

### **PrayerTimeTracker.css**

```css
.prayer-tracker {
  max-width: 500px;
  margin: 0 auto;
  padding: 30px;
  background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
  border-radius: 20px;
  color: white;
  box-shadow: 0 10px 40px rgba(0,0,0,0.3);
}

.prayer-tracker h1 {
  text-align: center;
  margin-bottom: 30px;
  font-size: 2em;
}

.current-time {
  text-align: center;
  background: rgba(255, 255, 255, 0.2);
  padding: 20px;
  border-radius: 15px;
  margin-bottom: 20px;
}

.current-time h2 {
  font-size: 3em;
  margin: 0;
  font-weight: bold;
}

.current-time p {
  margin: 10px 0 0 0;
  font-size: 1.1em;
}

.greeting {
  text-align: center;
  font-size: 1.5em;
  margin: 20px 0;
}

.next-prayer {
  background: rgba(255, 255, 255, 0.2);
  padding: 15px;
  border-radius: 10px;
  text-align: center;
  font-size: 1.2em;
  margin-bottom: 20px;
}

.prayer-times {
  background: rgba(255, 255, 255, 0.2);
  padding: 20px;
  border-radius: 15px;
}

.prayer-times h3 {
  text-align: center;
  margin-bottom: 15px;
}

.prayer-time {
  display: flex;
  justify-content: space-between;
  padding: 12px;
  border-bottom: 1px solid rgba(255, 255, 255, 0.3);
  font-size: 1.1em;
}

.prayer-time:last-child {
  border-bottom: none;
}
```

### **What We Used:**
✅ **Multiple useEffect hooks** for different purposes  
✅ **setInterval** for live clock  
✅ **Cleanup functions** to clear intervals  
✅ **Dependency arrays** to control when effects run  
✅ **Time calculations** and formatting  
✅ **Conditional logic** based on time  

---


## **Part 8: Tasks**

## 🎯 Task 1: Live Stopwatch

**Task:** Build a stopwatch that counts seconds

**Requirements:**
1. Create `Stopwatch.jsx`
2. Display time in format: MM:SS
3. Three buttons: Start, Pause, Reset
4. Use useEffect with setInterval
5. Clean up interval on unmount
6. Stop interval when paused

**Sample Output:**
```
⏱️ Stopwatch

02:35

[Start] [Pause] [Reset]
```

**Challenge:** Add lap time feature

---

## 🎯 Task 2: Document Title Updater

**Task:** Update browser tab title based on input

**Requirements:**
1. Create `TitleUpdater.jsx`
2. Input field for custom title
3. Use useEffect to update `document.title`
4. Show preview of title
5. Reset button to default title

**Sample Output:**
```
📝 Document Title Updater

Custom Title: [Type here...]

Preview: "React App - Type here..."

[Reset to Default]
```

**Challenge:** Add character counter (max 60)

---

## 🎯 Task 3: Countdown Timer

**Task:** Build a countdown timer for important events

**Requirements:**
1. Create `CountdownTimer.jsx`
2. Set a target date/time (e.g., Nigerian Independence Day)
3. Calculate and display: Days, Hours, Minutes, Seconds remaining
4. Update every second using useEffect
5. Show "Event Started!" when countdown reaches zero
6. Clean up interval

**Sample Output:**
```
🎉 Countdown to Independence Day

23 Days : 14 Hours : 32 Minutes : 45 Seconds

October 1, 2025
```

**Challenge:** Allow user to set custom target date

---


## **Part 9: Review**

### **Key Takeaways**
✅ useEffect handles side effects in React  
✅ Dependency array controls when effect runs:
  - No array = runs every render
  - Empty `[]` = runs once on mount
  - `[value]` = runs when value changes
✅ Return cleanup function to prevent memory leaks  
✅ Cleanup runs before re-running effect and on unmount  
✅ One useEffect per concern (separate effects for separate purposes)  

### **Common Mistakes**
- Forgetting dependency array (causing infinite loops)
- Not cleaning up timers/listeners
- Putting too much logic in one useEffect
- Modifying state directly in useEffect without proper dependencies
- Using index as dependency (use actual values)

### **Best Practices**
- Always clean up subscriptions, timers, and listeners
- Keep effects simple and focused
- List all dependencies used inside the effect
- Use multiple useEffect hooks for different concerns
- Name cleanup functions clearly

---

## **Next Lesson Preview**
Tomorrow we'll learn about **Working with APIs** - fetching data from external sources, handling loading states, and error management!